In [11]:
# !pip install --upgrade pip
# !pip install usefulfunctions

In [1]:
import os
import re
import numpy as np
import pandas as pd
import usefulfunctions
import pickle
import chardet  # Detecta encode do arquivo
from math import *
from unidecode import unidecode  # Útil para retirar cacteres especiais (como acentos) das palavras
from sklearn.utils import resample

In [2]:
def read_data(file_t):
    pd.set_option('display.max_columns', None)  # Mostrar todas as colunas
    pd.set_option('display.max_rows', 10)  # Mostrar apenas 10 linhas

    with open(file_t, 'rb') as check_raw:
        raw_data = check_raw.readline()
        encode = chardet.detect(raw_data).get('encoding')
        if encode == 'ascii':
            encode = 'utf-8'

    data = pd.read_csv(file_t, encoding=encode, sep=None, on_bad_lines='skip', engine='python')
    print('---------------------------------------')
    print('Base: {}'.format(file_t[6:-4]))
    print("Tamanho da base: {}".format(len(data)))

    del encode
    return data


def dates_filling(raw_df_t, *columns):
    if pd.isnull(raw_df_t.loc[columns[0]]) & pd.notnull(raw_df_t.loc[columns[1]]):
        raw_df_t[columns[0]] = raw_df_t[columns[1]]
    elif pd.notnull(raw_df_t.loc[columns[0]]) & pd.isnull(raw_df_t.loc[columns[1]]):
        raw_df_t[columns[1]] = raw_df_t[columns[0]]
    elif pd.isnull(raw_df_t.loc[columns[0]]) & pd.isnull(raw_df_t.loc[columns[1]]):
        raw_df_t[columns[0]] = pd.to_datetime('today', format='%Y-%m-%d')
        raw_df_t[columns[1]] = pd.to_datetime('today', format='%Y-%m-%d')

    return raw_df_t


# Padronizar o nome dos sintomas, tirando acentos e espaços.
def standarding_features(raw_df_t, *features, **kwargs):
    column = kwargs['key']

    if not raw_df_t[column].strip():
        for i in features:
            raw_df_t[i] = 0  # Remover aqui se for para não preencher com 0 em valores faltosos
        return raw_df_t
    else:
        for i in features:
            if re.search(i, unidecode(raw_df_t[column]).strip().lower()):  # Retira todos os caracteres especiais
                raw_df_t[i] = 1
            else:
                raw_df_t[i] = 0  # Remover aqui se for para não preencher com 0 em valores faltosos

    return raw_df_t


# ffdsfsdfsdfsd
def standarding_features_with_dict(raw_df_t, *column, **kwargs):
    kwargs = kwargs['key']

    if not raw_df_t[column[0]].strip():
        for key in kwargs.keys():
            if not raw_df_t[key]:
                raw_df_t[key] = 0  # Remover aqui se for para não preencher com 0 em valores faltosos
        return raw_df_t
    else:
        for key in kwargs.keys():
            for value in kwargs[key]:
                if raw_df_t[key] == 1:
                    break
                elif re.search(value, unidecode(str(raw_df_t[column[0]])).strip().lower()):  # Retira caract. especiais
                    raw_df_t[key] = 1
                    break
                else:
                    raw_df_t[key] = 0  # Remover aqui se for para não preencher com 0 em valores faltosos

    return raw_df_t


# Útil para salvar informações de agrupamento de uma coluna em arquivo txt
def save_value_counts_file(df_t, column_name_t):
    file_name = '{}.txt'.format(column_name_t)

    with open(file_name, 'w') as file_value:
        file_value.write(df_t[column_name_t].value_counts().to_string())


# Útil para adicionar colunas, função usada para adicionar as colunas de cada sintoma
def add_column(df_t, columns_t):
    for i, column in enumerate(columns_t):
        df_t.insert((i + 1), column, '', True)


# Útil para arredondar números, ainda mais em escalas inteiras
def round_up(number_t):
    decimal = (len(str(number_t)) * -1) + 1

    if decimal == 0:
        decimal = -1

    multiplier = 10 ** decimal
    final_number = ceil(number_t * multiplier) / multiplier

    return round(final_number)


# Salvando arquivo
def saving_data(df_t, file_t):
    df_t.to_csv(file_t, sep=';', encoding='utf-8', index=False)
    print("Salvando arquivo")
    print("Tamanho da base: {}".format(len(df_t)))


# Salvando o modelo da regressão
def saving_model(model_t, name_model_t):
    model_file = 'Modelos/{}.pkl'.format(name_model_t)

    with open(model_file, 'wb') as file:
        pickle.dump(model_t, file)


# Carregando o modelo da regressão
def loading_model(name_model_t):
    model_file = 'Modelos/{}.pkl'.format(name_model_t)

    with open(model_file, 'rb') as file:
        model_loaded = pickle.load(file)

    return model_loaded


# Balanceamento dos dados
def dataset_balancing(df_t, features_t, kind_t):
    if kind_t == 'Unbalanced':
        return df_t
    else:
        df_positives = df_t[df_t['resultadoTeste'] == 1]
        df_negatives = df_t[df_t['resultadoTeste'] == 0]

    if kind_t == 'Symptoms_Amount':
        df_positives = df_positives[df_positives[features_t].sum(axis=1) > 3]
        df_positives = df_positives[(df_positives['anosmia'] == 1) | (df_positives['hipogeusia'] == 1)]

    if len(df_positives) > len(df_negatives):
        df_majority = df_positives
        df_minority = df_negatives
    else:
        df_majority = df_negatives
        df_minority = df_positives

    df_size = round((len(df_minority) * 2) * 0.60)
    df_reduced = resample(df_majority, replace=True, n_samples=df_size, random_state=123)
    df_balanced = pd.concat([df_reduced, df_minority])

    return df_balanced


def dataset_conditions_balancing(df_t, cases_t):
    df_t = df_t[df_t['resultadoTeste'] == 1]

    df_case_1 = df_t[df_t['evolucaoCaso_{}'.format(cases_t[1])] == 0]
    df_case_2 = df_t[df_t['evolucaoCaso_{}'.format(cases_t[1])] == 1]

    if len(df_case_1) > len(df_case_2):
        df_majority = df_case_1
        df_minority = df_case_2
    else:
        df_majority = df_case_2
        df_minority = df_case_1

    df_size = round((len(df_minority) * 2) * 0.60)
    df_reduced = resample(df_majority, replace=True, n_samples=df_size, random_state=123)
    df_balanced = pd.concat([df_reduced, df_minority])

    return df_balanced


def get_files_names(base_dir_t, grouping=False):
    files = os.listdir(base_dir_t)
    files = [name for name in files if name[-3:] == 'csv']

    if grouping:
        files = grouping_datasets(files)

    return files


# Cria diretório para guardar dados
def directory_create(directory_list_t):
    for directory in directory_list_t:
        if not os.path.exists(directory):
            os.mkdir(directory)


def grouping_datasets(df_files_t):
    df = read_data('Bases/{}'.format(df_files_t[0]))

    if len(df_files_t) == 1:
        return df_files_t
    else:
        for i in range(1, len(df_files_t)):
            df_temp = read_data('Bases/{}'.format(df_files_t[i]))
            df = pd.concat([df, df_temp], ignore_index=True)
            print('Tamanho da base concatenada: {}'.format(len(df)))

        name_temp = df_files_t[0][:3]

        for dataset in df_files_t:
            os.remove('Bases/{}'.format(dataset))

        saving_data(df, 'Bases/{}ALL.csv'.format(name_temp))

        df_files_t = os.listdir('Bases/')
        df_files_t = [name for name in df_files_t if name[-3:] == 'csv']

        return df_files_t


def clean_features(df_t, datas_t, symptoms_t=False, conditions_t=False):
    if symptoms_t:
        df_t.drop(df_t[(df_t[datas_t['symptoms']] == 0).all(axis=1)].index, inplace=True)

    if conditions_t:
        df_t.drop(df_t[(df_t[datas_t['conditions']] == 0).all(axis=1)].index, inplace=True)


def convert_to_categorical(features_t):
    ages = [12, 18, 30, 45, 60, 75, 90, 121]
    days = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90]
    categories = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
    # categories = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S']

    for i, age in enumerate(ages):
        if features_t['idade'] < age:
            features_t['idade'] = categories[i]
            break

    for i, days in enumerate(days):
        if features_t['diasSintomas'] <= days:
            features_t['diasSintomas'] = categories[i]
            break

    return features_t

In [4]:
def preprocessing(file_t):
    df = read_data(file_t)

    # Colunas para analisar os dados e deletar as amostras (rows)
    columns_analyze = ['evolucaoCaso', 'sintomas', 'tipoTeste', 'resultadoTeste', 'classificacaoFinal']

    # Colunas para deletar
    columns_del = ['id', 'estadoNotificacao', 'estadoNotificacaoIBGE', 'municipioNotificacao',
                   'municipioNotificacaoIBGE', 'profissionalSaude', 'profissionalSeguranca', 'cbo', 'sexo', 'racaCor',
                   'estado', 'estadoIBGE', 'municipio', 'municipioIBGE', 'dataNotificacao', 'dataInicioSintomas',
                   'estadoTeste', 'dataTeste', 'dataEncerramento', 'cnes']

    # Sintomas e condições para virarem colunas
    datas = {'other_symptoms': {'mialgia': ['mialgia', 'corpo', 'muscular'],
                                'cabeca': ['cefaleia', 'cabeca'],
                                'fadiga': ['fadiga', 'cansaco', 'fraqueza', 'indisposicao', 'adinamia', 'moleza',
                                           'astenia'],
                                'nauseas': ['nausea', 'nauseas', 'tontura', 'mal estar', 'enjoo'],
                                'coriza': ['coriza'],
                                'anosmia': ['olfato', 'anosmia', 'hiposmia'],
                                'hipogeusia': ['paladar', 'hipogeusia', 'disgeusia'],
                                'diarreia': ['diarreia'], 'febre': ['febre', 'febril'],
                                'tosse': ['tosse']
                                },
             'symptoms': ['tosse', 'febre', 'garganta', 'dispneia', 'cabeca', 'coriza', 'hipogeusia', 'anosmia',
                          'fadiga', 'nauseas', 'mialgia', 'diarreia'],
             'conditions': ['cardiacas', 'diabetes', 'respiratorias', 'renais', 'imunologica', 'obesidade',
                            'imunossupressao', 'gestante', 'puerpera']
             }

    # Deletando alguns rows com base nos valores nulos ou diferentes do necessário para se trabalhar
    df.dropna(how='any', subset=columns_analyze, inplace=True)
    df['tipoTeste'].replace(to_replace='TESTE RÁPIDO - ANTÍGENO', value='RT-PCR', inplace=True)
    df.drop(df[df['tipoTeste'] != 'RT-PCR'].index, inplace=True)
    df.drop(df[~df['classificacaoFinal'].str.contains(r'Confirmado')].index, inplace=True)
    df.drop(df[df['resultadoTeste'].str.contains(r'^[In].*')].index, inplace=True)
    df.drop(df[(df['evolucaoCaso'] == 'Cancelado') | (df['evolucaoCaso'] == 'Ignorado')].index, inplace=True)
    del columns_analyze

    # Correção, muda valores de null para empty strings.
    df['condicoes'].fillna('', inplace=True)
    df['outrosSintomas'].fillna('', inplace=True)

    # Consertando um probleminha de datas com valores ausentes para poder aplicar funções
    df = df.apply(dates_filling, args=['dataNotificacao', 'dataInicioSintomas'], axis=1)
    data_notificacao = pd.to_datetime(df['dataNotificacao'], errors='coerce', format='%Y-%m-%d')
    data_inicio_sintomas = pd.to_datetime(df['dataInicioSintomas'], errors='coerce', format='%Y-%m-%d')
    df.insert(0, "diasSintomas", (data_notificacao - data_inicio_sintomas).dt.days, True)
    df.loc[df['diasSintomas'] < 0, 'diasSintomas'] = df['diasSintomas'] * -1
    df.drop(df[df['diasSintomas'] > 90].index, inplace=True)

    # Deletando atributos irrelevantes para análise
    df.drop(columns_del, axis=1, inplace=True)
    del columns_del
    del data_inicio_sintomas, data_notificacao

    # Adiciona os sintomas e condicoes como colunas no dataframe, cada sintoma/condição vira 1 coluna
    add_column(df, datas['symptoms'] + [col for col in datas['other_symptoms'].keys() if col not in datas['symptoms']] +
               datas['conditions'])

    # Preenche as colunas dos sintomas e condicoes com 1
    for feature, column in zip([datas['symptoms'], datas['conditions']], ['sintomas', 'condicoes']):
        df = df.apply(standarding_features, axis=1, args=feature, key=column)

    df = df.apply(standarding_features_with_dict, axis=1, args=['outrosSintomas'], key=datas['other_symptoms'])
    df.drop(['sintomas', 'condicoes', 'outrosSintomas'], axis=1, inplace=True)
    df.drop(df[(df[datas['symptoms']] == 0).all(axis=1)].index, inplace=True)

    # Trocar alguns valores
    cols_change = ['resultadoTeste', 'evolucaoCaso']  # Índice 0 no zip
    old_values = [['Negativo', 'Positivo'], [r'.*tratamento.*', r'Internado.*']]  # Índice 1 no zip
    new_values = [[0, 1], ['Cura', 'Internado']]  # Índice 2 no zip
    use_regex = [False, True]  # Índice 3 no zip

    for values in zip(cols_change, old_values, new_values, use_regex):
        df[values[0]].replace(to_replace=values[1][0], value=values[2][0], regex=values[3], inplace=True)
        df[values[0]].replace(to_replace=values[1][1], value=values[2][1], regex=values[3], inplace=True)

    del cols_change, old_values, new_values, use_regex

    for col in ['idade', 'diasSintomas']:
        df.drop(df[df[col].isin([np.nan, np.inf, -np.inf])].index, inplace=True)
        df[col] = df[col].astype(int)

    # Salvando o arquivo
    saving_data(df, file_t)

    return df

In [5]:
df = preprocessing("../../database/dadosRN2.csv")

---------------------------------------
Base: database/dadosRN2
Tamanho da base: 511433
Salvando arquivo
Tamanho da base: 2750


In [6]:
df.head()

,diasSintomas,tosse,febre,garganta,dispneia,cabeca,coriza,hipogeusia,anosmia,fadiga,nauseas,mialgia,diarreia,cardiacas,diabetes,respiratorias,renais,imunologica,obesidade,imunossupressao,gestante,puerpera,tipoTeste,resultadoTeste,classificacaoFinal,evolucaoCaso,idade
28,70,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,RT-PCR,1,Confirmado Laboratorial,Cura,51
140,1,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,RT-PCR,1,Confirmado Laboratorial,Cura,25
205,4,1,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,RT-PCR,1,Confirmado Laboratorial,Cura,38
290,3,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,RT-PCR,1,Confirmado Laboratorial,Cura,58
571,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,RT-PCR,1,Confirmado Laboratorial,Cura,28


In [7]:
write = pd.ExcelWriter('covid_rn_sus.xlsx')
df.to_excel(write, 'dados', index = False)
write.save()

# Visualizando os Dados

In [53]:
covidData = pd.read_excel("../../database/covid_rn_sus.xlsx")

In [54]:
covidData.head()

,diasSintomas,tosse,febre,garganta,dispneia,cabeca,coriza,hipogeusia,anosmia,fadiga,...,imunologica,obesidade,imunossupressao,gestante,puerpera,tipoTeste,resultadoTeste,classificacaoFinal,evolucaoCaso,idade
0,70,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,RT-PCR,1,Confirmado Laboratorial,Cura,51
1,1,1,1,1,0,0,1,0,0,0,...,0,0,0,0,0,RT-PCR,1,Confirmado Laboratorial,Cura,25
2,4,1,1,0,0,1,0,0,0,1,...,0,0,0,0,0,RT-PCR,1,Confirmado Laboratorial,Cura,38
3,3,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,RT-PCR,1,Confirmado Laboratorial,Cura,58
4,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,RT-PCR,1,Confirmado Laboratorial,Cura,28


In [34]:
covidData['evolucaoCaso'].value_counts()

Cura         2512
Óbito         228
Internado      10
Name: evolucaoCaso, dtype: int64

In [35]:
covidData['classificacaoFinal'].value_counts()

Confirmado Laboratorial              2565
Confirmado por Critério Clínico       158
Confirmado Clínico-Epidemiológico      25
Confirmado Clínico-Imagem               2
Name: classificacaoFinal, dtype: int64

In [36]:
covidData['tipoTeste'].value_counts()

RT-PCR    2750
Name: tipoTeste, dtype: int64

In [37]:
covidData['resultadoTeste'].value_counts()

1    2426
0     324
Name: resultadoTeste, dtype: int64

In [38]:
covidData = covidData.drop(columns=["diasSintomas", "tipoTeste", "classificacaoFinal"])
covidData.head()

,tosse,febre,garganta,dispneia,cabeca,coriza,hipogeusia,anosmia,fadiga,nauseas,...,respiratorias,renais,imunologica,obesidade,imunossupressao,gestante,puerpera,resultadoTeste,evolucaoCaso,idade
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,Cura,51
1,1,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,Cura,25
2,1,1,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,Cura,38
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,Cura,58
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,Cura,28


In [39]:
dadosSemCovid = covidData.loc[(covidData['resultadoTeste'] != 1)]
covidData = covidData.drop(dadosSemCovid.index)
covidData['resultadoTeste'].value_counts()

1    2426
Name: resultadoTeste, dtype: int64

In [40]:
dadosInternados = covidData.loc[(covidData['evolucaoCaso'] == 'Internado')]
covidData = covidData.drop(dadosInternados.index)
covidData['evolucaoCaso'].value_counts()

Cura     2189
Óbito     227
Name: evolucaoCaso, dtype: int64

In [41]:
for i in range (0, 12):
    covidData['idade'].replace(i, 'CRIANCA', inplace = True)
    
for i in range (12, 18):
    covidData['idade'].replace(i, 'JOVEM', inplace = True)
    
for i in range (18, 30):
    covidData['idade'].replace(i, 'ADULTO1', inplace = True)

for i in range (30, 45):
    covidData['idade'].replace(i, 'ADULTO2', inplace = True)

for i in range (45, 60):
    covidData['idade'].replace(i, 'ADULTO3', inplace = True)

for i in range (60, 75):
    covidData['idade'].replace(i, 'IDOSO1', inplace = True)

for i in range (75, 90):
    covidData['idade'].replace(i, 'IDOSO2', inplace = True)

for i in range (90, 115):
    covidData['idade'].replace(i, 'IDOSO3', inplace = True)

In [42]:
covidData['idade'].value_counts()

ADULTO2    902
ADULTO3    522
ADULTO1    433
IDOSO1     286
IDOSO2     131
JOVEM       63
CRIANCA     48
IDOSO3      31
Name: idade, dtype: int64

In [43]:
covidData.head()

,tosse,febre,garganta,dispneia,cabeca,coriza,hipogeusia,anosmia,fadiga,nauseas,...,respiratorias,renais,imunologica,obesidade,imunossupressao,gestante,puerpera,resultadoTeste,evolucaoCaso,idade
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,Cura,ADULTO3
1,1,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,Cura,ADULTO1
2,1,1,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,Cura,ADULTO2
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,Cura,ADULTO3
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,Cura,ADULTO1


In [44]:
dadosObitos = covidData.loc[(covidData['evolucaoCaso'] == 'Óbito')]
dadosObitos.shape

(227, 24)

In [45]:
dadosCurados = covidData.loc[(covidData['evolucaoCaso'] == 'Cura')]
dadosCurados.shape

(2189, 24)

In [46]:
selecionados = dadosCurados.sample(757)
selecionados.shape

(757, 24)

In [47]:
balanceados = pd.merge(dadosObitos, selecionados, how = 'outer')
balanceados.shape

(984, 24)

In [48]:
import sklearn
covidData_shuffled = sklearn.utils.shuffle(balanceados)
covidData_shuffled.head()

,tosse,febre,garganta,dispneia,cabeca,coriza,hipogeusia,anosmia,fadiga,nauseas,...,respiratorias,renais,imunologica,obesidade,imunossupressao,gestante,puerpera,resultadoTeste,evolucaoCaso,idade
674,1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,Cura,ADULTO3
944,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,Cura,ADULTO3
312,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,Cura,ADULTO3
574,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,Cura,ADULTO3
647,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,Cura,ADULTO1


In [49]:
covidData_shuffled.shape

(984, 24)

In [50]:
covidData_shuffled.reset_index(inplace=True, drop = True)
covidData_shuffled.head()

,tosse,febre,garganta,dispneia,cabeca,coriza,hipogeusia,anosmia,fadiga,nauseas,...,respiratorias,renais,imunologica,obesidade,imunossupressao,gestante,puerpera,resultadoTeste,evolucaoCaso,idade
0,1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,Cura,ADULTO3
1,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,Cura,ADULTO3
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,Cura,ADULTO3
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,Cura,ADULTO3
4,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,Cura,ADULTO1


In [51]:
# covidData_shuffled['resultadoTeste'].value_counts()
covidData = covidData.drop(columns=["resultadoTeste"])
covidData.head()

,tosse,febre,garganta,dispneia,cabeca,coriza,hipogeusia,anosmia,fadiga,nauseas,...,diabetes,respiratorias,renais,imunologica,obesidade,imunossupressao,gestante,puerpera,evolucaoCaso,idade
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Cura,ADULTO3
1,1,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,Cura,ADULTO1
2,1,1,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,Cura,ADULTO2
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Cura,ADULTO3
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Cura,ADULTO1


In [52]:
write = pd.ExcelWriter('rn_sus_70_30.xlsx')
covidData_shuffled.to_excel(write, 'dados', index = False)
write.save()